\
            # 51. MongoDB 基础（文档数据库）（MongoDB Fundamentals）

            目标：掌握 MongoDB 的核心概念与常见 CRUD/索引/聚合用法，并能写出可维护的数据模型与查询。
本章包含可选依赖 `pymongo` 的真实连接示例；如果本机没有 MongoDB，会自动降级到纯 Python 的“文档集合模拟器”，保证代码块可运行。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 字典/列表/JSON
- 异常处理
- 数据库基本概念（索引/查询）


## 知识点地图

- 1. 核心概念：文档/集合/数据库、BSON 与 ObjectId
- 2. CRUD：insert/find/update/delete（以 pymongo 语义讲解）
- 3. 索引：让查询快，但写入更慢
- 4. 聚合管道：match/group/project/sort 的套路
- 5. Schema 设计：嵌入 vs 引用、避免增长无界
- 6. 可运行：纯 Python 文档集合模拟器（教学用）
- 7. 可选：连接真实 MongoDB（pymongo）


## 自检清单（学完打勾）

- [ ] 理解文档（document）/集合（collection）/数据库（database）与 BSON/JSON 的关系
- [ ] 会做基础 CRUD（insert/find/update/delete）并理解过滤条件
- [ ] 知道 ObjectId 的作用与常见坑（字符串/时间顺序）
- [ ] 理解索引的意义与代价，知道常见索引类型（单字段/复合/唯一）
- [ ] 理解聚合（aggregation pipeline）的基本思路（match/group/project/sort）
- [ ] 具备基本的 schema 设计能力：嵌入 vs 引用、增长数组、热字段


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：核心概念：文档/集合/数据库、BSON 与 ObjectId

- MongoDB 以 **文档** 为核心：一条记录就是一个 JSON-like 对象（实际存 BSON）。
- 文档放在 **集合**（collection）里；集合属于 **数据库**。
- 主键 `_id` 默认是 **ObjectId**（12 字节，包含时间戳等信息）。

常见坑：
- `_id` 在代码里常被当作字符串传来传去，查询时类型不匹配导致查不到。
- 文档模式灵活 ≠ 不需要约束：仍建议在应用层/数据库层做校验与约束（例如唯一索引）。


## 知识点 2：CRUD：insert/find/update/delete（以 pymongo 语义讲解）

- 插入：insert_one/insert_many
- 查询：find/find_one（过滤条件是 dict）
- 更新：update_one/update_many（使用 `$set`、`$inc` 等更新操作符）
- 删除：delete_one/delete_many

查询条件例子：
- `{"age": {"$gte": 18}}`
- `{"tags": "python"}`（数组包含）

注意：更新务必使用 `$set`，不要直接把整条文档覆盖掉（除非你就是要 replace）。


## 知识点 3：索引：让查询快，但写入更慢

- 索引让查询更快，但会带来：写入/更新更慢、占空间、维护成本。
- 常见索引：
  - 单字段索引（最常用）
  - 复合索引（注意最左前缀）
  - 唯一索引（保证业务唯一性）

实践建议：
- 以“查询模式”驱动索引设计：先看最常用的过滤条件与排序。
- 对低选择性字段（只有几个取值）单独建索引往往收益不大。


## 知识点 4：聚合管道：match/group/project/sort 的套路

聚合管道（aggregation pipeline）可以理解为“可组合的数据处理流水线”：
- `$match`：过滤（越早越好）
- `$group`：分组统计
- `$project`：字段选择/重命名/计算
- `$sort`：排序
- `$limit`：限制结果

建议：先写清楚需求，再逐步堆 pipeline，最后看 explain/执行时间。


## 知识点 5：Schema 设计：嵌入 vs 引用、避免增长无界

MongoDB 的 schema 设计关键取决于访问模式：
- **嵌入（embed）**：读取一次拿全（读快），但更新局部可能复杂；文档大小有限制。
- **引用（reference）**：拆表风格（更新更独立），但读需要额外查询/聚合。

常见坑：
- “无限增长数组”（例如把所有日志都 append 到同一个文档）会导致文档膨胀与性能问题。
- 热点字段高频更新会造成写热点，可能需要拆分或分片策略（进阶）。


## 知识点 6：可运行：纯 Python 文档集合模拟器（教学用）

为了保证在没有 MongoDB 的环境也能跑通，本段实现一个极简“集合”模拟：
- insert/find/update/delete
- 支持最常见的过滤：等值、$gte/$lte
- 支持最常见的更新：$set/$inc

这不是 MongoDB，但足以让你练习“查询条件/更新语义/聚合思路”。


In [ ]:
import copy
import time
import uuid


def _now_ms():
    return int(time.time() * 1000)


def _match(doc, filt: dict) -> bool:
    for k, cond in (filt or {}).items():
        v = doc.get(k)
        if isinstance(cond, dict):
            for op, x in cond.items():
                if op == '$gte' and not (v >= x):
                    return False
                if op == '$lte' and not (v <= x):
                    return False
                if op == '$gt' and not (v > x):
                    return False
                if op == '$lt' and not (v < x):
                    return False
        else:
            if v != cond:
                return False
    return True


def _apply_update(doc, upd: dict):
    for op, payload in (upd or {}).items():
        if op == '$set':
            for k, v in payload.items():
                doc[k] = v
        elif op == '$inc':
            for k, v in payload.items():
                doc[k] = (doc.get(k) or 0) + v
        else:
            raise ValueError('unsupported update op: ' + op)


class MiniCollection:
    def __init__(self):
        self._docs = []

    def insert_one(self, doc: dict):
        d = copy.deepcopy(doc)
        d.setdefault('_id', uuid.uuid4().hex[:8])
        d.setdefault('created_at', _now_ms())
        self._docs.append(d)
        return d['_id']

    def find(self, filt=None):
        for d in self._docs:
            if _match(d, filt or {}):
                yield copy.deepcopy(d)

    def find_one(self, filt=None):
        return next(self.find(filt), None)

    def update_one(self, filt, update):
        for d in self._docs:
            if _match(d, filt or {}):
                _apply_update(d, update)
                return 1
        return 0

    def delete_one(self, filt):
        for i, d in enumerate(self._docs):
            if _match(d, filt or {}):
                self._docs.pop(i)
                return 1
        return 0


col = MiniCollection()
col.insert_one({'name': 'Alice', 'age': 18})
col.insert_one({'name': 'Bob', 'age': 20})
col.insert_one({'name': 'Carol', 'age': 17})

print('age>=18:', [d['name'] for d in col.find({'age': {'$gte': 18}})])
print('find_one Bob:', col.find_one({'name': 'Bob'}))

col.update_one({'name': 'Alice'}, {'$inc': {'age': 1}, '$set': {'vip': True}})
print('Alice after update:', col.find_one({'name': 'Alice'}))

col.delete_one({'name': 'Carol'})
print('all:', [d['name'] for d in col.find({})])


## 知识点 7：可选：连接真实 MongoDB（pymongo）

如果你本机有 MongoDB：
1) 安装驱动：`pip install pymongo`
2) 设置环境变量 `MONGODB_URI`（例如 `mongodb://localhost:27017`）

下面示例会尝试连接；失败会给出提示，不影响本章其它代码运行。


In [ ]:
import os

uri = os.getenv('MONGODB_URI')
try:
    from pymongo import MongoClient
except Exception as e:
    print('pymongo not available:', type(e).__name__, e)
    print('install: pip install pymongo')
else:
    if not uri:
        print('MONGODB_URI not set, example: mongodb://localhost:27017')
    else:
        try:
            client = MongoClient(uri, serverSelectionTimeoutMS=1000)
            client.admin.command('ping')
            db = client['demo_db']
            col = db['users']
            col.insert_one({'name': 'Alice', 'age': 18})
            print(list(col.find({'age': {'$gte': 18}}, {'_id': 0})))
        except Exception as e:
            print('connect failed:', type(e).__name__, e)


## 常见坑

- 把 _id 当字符串传递但查询时不转换类型（ObjectId）
- 更新不使用 $set，误把整条文档覆盖
- 索引乱建：写入变慢、占空间、维护复杂
- 无限增长数组/单文档过大：导致性能与存储问题
- 缺少唯一约束：并发下出现重复数据


## 综合小案例：设计一个“用户 + 订单”数据模型（嵌入 vs 引用）

请基于访问模式做设计：
- 需求：用户页面需要显示最近 5 笔订单摘要；订单详情页需要完整订单行项目。

请给出两种方案：
1) 嵌入（用户文档里存订单摘要）
2) 引用（orders 集合单独存，通过 user_id 关联）

并写出：
- 你会给哪些字段建索引？为什么？
- 如何避免订单行项目无限增长导致单文档过大？


In [ ]:
# 这是设计题，建议用 Markdown 画出文档结构与索引。
# 本 cell 不运行代码。


## 自测题（不写代码也能回答）

- MongoDB 的 document/collection/db 分别是什么？
- 为什么更新要用 $set/$inc？直接替换有什么风险？
- 索引为什么会让写入变慢？
- 嵌入 vs 引用如何选择？核心依据是什么？
- 聚合管道的基本套路是什么？match 应该放前还是放后？


## 练习题（建议写代码）

- 用 MiniCollection 扩展支持 $in 操作符，并写 3 个查询例子。
- 实现一个简单 aggregation：按 age 分组统计人数（group）。
- 如果使用真实 MongoDB：为 users.age 建索引，并写 explain 对比（了解）。
